In [22]:
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import json


PROJ_DIR = Path('/network/scratch/a/aasheesh.singh/quick-recipe/')
DATA_DIR = PROJ_DIR / 'data'
YOUCOOK_DIR = PROJ_DIR / 'youcook2'

TRAIN_FRAC = 0.7
# RANDOM_SEED = 23456
# np.random.seed(RANDOM_SEED)

In [23]:
df = pd.read_csv(str(YOUCOOK_DIR / 'reviewed_0812.csv'))

In [24]:
indices = list(range(len(df)))
np.random.shuffle(indices)
target_train_len = int(TRAIN_FRAC * len(df))

In [25]:
target_train_len, len(df) - target_train_len

(10857, 4654)

In [26]:
count_df = df[['VideoUrl', 'Title']].groupby('VideoUrl').agg(num_rows=('Title', 'count'))
count_df

,num_rows
VideoUrl,
https://www.youtube.com/watch?v=-CP9gKR9GR4,46
https://www.youtube.com/watch?v=-ErPSunMfcs,34
https://www.youtube.com/watch?v=-Ju39A-G0Dk,70
https://www.youtube.com/watch?v=-ORd4EBliqw,75
https://www.youtube.com/watch?v=-dh_uGahzYo,66
...,...
https://www.youtube.com/watch?v=zFKhct4g8as,53
https://www.youtube.com/watch?v=zLBRrWd4DTo,37
https://www.youtube.com/watch?v=zYMelEXKars,34


In [27]:
video_urls = df['VideoUrl'].unique()

In [34]:
video_url_indices = [i for i in range(len(video_urls))]

In [35]:
np.random.shuffle(video_url_indices)
print(video_url_indices[:4])
train_size = 0
url_idx = 0

while train_size < target_train_len:
    train_size += count_df.loc[video_urls[video_url_indices[url_idx]]]['num_rows']
    url_idx += 1

[113, 232, 175, 102]


In [36]:
train_size, len(df) - train_size, train_size/len(df)

(10860, 4651, 0.7001482818644833)

In [37]:
train_indices, val_indices = video_url_indices[:url_idx], video_url_indices[url_idx:]

In [38]:
train_urls, val_urls = video_urls[train_indices], video_urls[val_indices]

In [39]:
train_urls

array(['https://www.youtube.com/watch?v=k38Al8giI-U',
       'https://www.youtube.com/watch?v=6uB1pcyBCBw',
       'https://www.youtube.com/watch?v=91z7e22XGy8',
       'https://www.youtube.com/watch?v=0B-59Ok_r1Y',
       'https://www.youtube.com/watch?v=LqCDIssigHo',
       'https://www.youtube.com/watch?v=PYjrGqPHGhY',
       'https://www.youtube.com/watch?v=BGCW_PWhY9g',
       'https://www.youtube.com/watch?v=NWBh5-CfPHk',
       'https://www.youtube.com/watch?v=oR2QDpoatcQ',
       'https://www.youtube.com/watch?v=2UY6Fztzpng',
       'https://www.youtube.com/watch?v=_Vw0jI7kJtY',
       'https://www.youtube.com/watch?v=6gObQR5Vm4M',
       'https://www.youtube.com/watch?v=02nUKT0A7uE',
       'https://www.youtube.com/watch?v=E_0tikIOFIc',
       'https://www.youtube.com/watch?v=QepadF9vOdA',
       'https://www.youtube.com/watch?v=z_ZHVTqbSZo',
       'https://www.youtube.com/watch?v=UbunZxDifYU',
       'https://www.youtube.com/watch?v=UzqlXpAAAWk',
       'https://www.youtube.

In [43]:
import pickle

In [52]:
split_df = pd.DataFrame({'VideoUrl': np.concatenate([train_urls, val_urls]), 'Split': ['train' for i in range(len(train_urls))] + ['val' for i in range(len(val_urls))]})
split_df

,VideoUrl,Split
0,https://www.youtube.com/watch?v=k38Al8giI-U,train
1,https://www.youtube.com/watch?v=6uB1pcyBCBw,train
2,https://www.youtube.com/watch?v=91z7e22XGy8,train
3,https://www.youtube.com/watch?v=0B-59Ok_r1Y,train
4,https://www.youtube.com/watch?v=LqCDIssigHo,train
...,...,...
347,https://www.youtube.com/watch?v=nWGpCmDlNU4,val
348,https://www.youtube.com/watch?v=OIHLdm9BuoU,val
349,https://www.youtube.com/watch?v=_Vzpj0cXoSM,val
350,https://www.youtube.com/watch?v=pCTdsgv1wZ4,val


In [53]:
split_df.to_csv(str(DATA_DIR / 'train_val_split.csv'), index=False)

PermissionError: [Errno 13] Permission denied: '/network/scratch/a/aasheesh.singh/quick-recipe/data/train_val_split.csv'

In [54]:
!ls

1.0_Data_Preprocessing.ipynb  2.1_BERT_Knowledge_Extraction.ipynb
1.1_Train_Test_Split.ipynb    transformer-key-step.ipynb


In [56]:
split_df.to_csv('train_val_split.csv', index=False)

In [57]:
_df = pd.read_csv('train_val_split.csv')

In [58]:
_df

,VideoUrl,Split
0,https://www.youtube.com/watch?v=k38Al8giI-U,train
1,https://www.youtube.com/watch?v=6uB1pcyBCBw,train
2,https://www.youtube.com/watch?v=91z7e22XGy8,train
3,https://www.youtube.com/watch?v=0B-59Ok_r1Y,train
4,https://www.youtube.com/watch?v=LqCDIssigHo,train
...,...,...
347,https://www.youtube.com/watch?v=nWGpCmDlNU4,val
348,https://www.youtube.com/watch?v=OIHLdm9BuoU,val
349,https://www.youtube.com/watch?v=_Vzpj0cXoSM,val
350,https://www.youtube.com/watch?v=pCTdsgv1wZ4,val
